In [2]:
import pandas as pd
from modules.KIS import public_api, stock
API = public_api.Public_API()
import datetime
import time
import FinanceDataReader as fdr
import pickle
from pprint import pprint
import os
from tqdm import tqdm
import pymysql

In [3]:
def save_price_info(res, stock_info):
    if res["rt_cd"] == "0":
        stock_code = res["output1"]["stck_shrn_iscd"]
        for j in range(len(res["output2"])):
            try:
                Date = res["output2"][j]["stck_bsop_date"]
                Close = res["output2"][j]["stck_clpr"]
                Open  = res["output2"][j]["stck_oprc"]
                High = res["output2"][j]["stck_hgpr"]
                Low = res["output2"][j]["stck_lwpr"]
                Volume = res["output2"][j]["acml_vol"]
                Amount = res["output2"][j]["acml_tr_pbmn"]
                Change = res["output2"][j]["prdy_vrss"]

                stock_info.loc[len(stock_info)] = [stock_code, Date, Open, High, Low, Close, Volume, Amount, Change]
            except:
                continue
            
    else:
        print(f"Error: {res['rt_cd']}, {res['msg_cd']}", {res['msg1']})
    
    return stock_info
        
    
        
    

    

In [5]:
stock_code_list = stock.get_stock_list()
#숫자로 이루어진 종목코드만 추출
stock_code_list = [x for x in stock_code_list if x.isdigit()]

for stock_code in tqdm(stock_code_list):
    if os.path.exists(f"/app/modules/KIS/Data/stock_price/{stock_code}.csv"):
        continue
    try:
        print(stock_code)
        stock_info = pd.DataFrame(columns=["Stock_Code","Date", "Open", "High", "Low", "Close", "Volume", "Amount", "Change"])
        today = datetime.datetime.today()
        str_today = today.strftime("%Y%m%d")
        first_date = (datetime.datetime.strptime(str_today, "%Y%m%d") - datetime.timedelta(days=140)).strftime("%Y%m%d")
        listing_date = stock.get_listing_date(stock_code)
        while datetime.datetime.strptime(str_today, "%Y%m%d") > datetime.datetime.strptime(str(listing_date), "%Y%m%d"):
            res = API.국내주식기간별시세(stock_code, first_date, str_today)
            time.sleep(0.05)
            history_len = len(stock_info)
            stock_info = save_price_info(res, stock_info)
            if stock_info is False:
                break
            if res["rt_cd"] == "0":
                str_today = first_date
                first_date = (datetime.datetime.strptime(str_today, "%Y%m%d") - datetime.timedelta(days=100)).strftime("%Y%m%d")
            else:
                break
            if history_len == len(stock_info):
                break
    except:
        pass
    stock_info.to_csv(f"/app/modules/KIS/Data/stock_price/{stock_code}.csv", index=False)
                
    
    
    
    

    

  0%|          | 0/3241 [00:00<?, ?it/s]

356680


 45%|████▍     | 1457/3241 [00:00<00:00, 4052.56it/s]

442900
446150


 57%|█████▋    | 1863/3241 [00:01<00:00, 1532.06it/s]

...

005320


 59%|█████▊    | 1900/3241 [06:16<27:23,  1.23s/it]

005360


 59%|█████▊    | 1901/3241 [07:14<41:49,  1.87s/it]

005380


In [2]:
connection = pymysql.connect(host='daisnas.synology.me',
                             user='dns05018',
                             password='ThlSLoVE77&&',
                             database='AutoMLStockTrader',
                             port=18310)

                             

In [ ]:
stock_code_list = stock.get_stock_list()
#숫자로 이루어진 종목코드만 추출
stock_code_list = [x for x in stock_code_list if x.isdigit()]

with connection.cursor() as cursor:
    
    for stock_code in tqdm(stock_code_list):
        stock_df = pd.read_csv(f"/app/modules/KIS/Data/stock_price/{stock_code}.csv")
        stock_df = stock_df.sort_values(by="Date", ascending=True)
        stock_df = stock_df.reset_index(drop=True)
        stock_df["Date"] = stock_df["Date"].apply(lambda x: datetime.datetime.strptime(str(x), "%Y%m%d"))
        stock_df["Date"] = stock_df["Date"].apply(lambda x: x.strftime("%Y-%m-%d"))
        stock_df.columns = ["stock_code", "date", "open", "high", "low", "close", "volume", "amount", "price_change"]
        #컬럼 순서 변경
        stock_df = stock_df[["date", "stock_code", "open", "high", "low", "close", "volume", "amount", "price_change"]]

        
        
    
    
    
    
    
    

In [ ]:
stock_code_list = stock.get_stock_list()
#숫자로 이루어진 종목코드만 추출
stock_code_list = [x for x in stock_code_list if x.isdigit()]
stock_code_list = []
with connection.cursor() as cursor:
    
    for stock_code in tqdm(stock_code_list):
        stock_df = pd.read_csv(f"/app/modules/KIS/Data/stock_price/{stock_code}.csv")
        stock_df.drop_duplicates(subset=["Date"], inplace=True)
        stock_df = stock_df.sort_values(by="Date", ascending=True)
        stock_df = stock_df.reset_index(drop=True)
        stock_df["Date"] = stock_df["Date"].apply(lambda x: datetime.datetime.strptime(str(x), "%Y%m%d"))
        stock_df["Date"] = stock_df["Date"].apply(lambda x: x.strftime("%Y-%m-%d"))
        """stock_df.columns = ["stock_code", "date", "open", "high", "low", "close", "volume", "amount", "price_change"]
        #컬럼 순서 변경
        stock_df = stock_df[["date", "stock_code", "open", "high", "low", "close", "volume", "amount", "price_change"]]"""
        for i in range(len(stock_df)):
            sql = '''
            INSERT INTO KOREA_STOCK_PRICE (date, stock_code, open, high, low, close, volume, amount, price_change)
            VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)
            '''
            cursor.execute(sql, (stock_df["Date"][i], str(stock_df["Stock_Code"][i]), stock_df["Open"][i], stock_df["High"][i], stock_df["Low"][i], stock_df["Close"][i], stock_df["Volume"][i], stock_df["Amount"][i], stock_df["Change"][i]))
        connection.commit()
        
connection.close()
        
    
    
    
    
    
    

In [3]:
#KOREA_STOCK_PRICE 테이블의 내용을 모두 삭제
with connection.cursor() as cursor:
    sql = '''
    DELETE FROM KOREA_STOCK_PRICE
    '''
    cursor.execute(sql)
    connection.commit()
    

In [4]:
stock_code_list = stock.get_stock_list()
#숫자로 이루어진 종목코드만 추출
stock_code_list = [x for x in stock_code_list if x.isdigit()]
for stock_code in tqdm(stock_code_list):
    if os.path.exists(f"/app/modules/KIS/Data/stock_price/{stock_code}.csv"):
        df = pd.read_csv(f"/app/modules/KIS/Data/stock_price/{stock_code}.csv")
        #중복된 데이터 삭제
        df.drop_duplicates(subset=["Date"], inplace=True)
        #10줄 이하면 .csv 파일 삭제
        if len(df) < 10:
            os.remove(f"/app/modules/KIS/Data/stock_price/{stock_code}.csv")
            print(f"{stock_code} 삭제")

 49%|████▉     | 1585/3241 [00:05<00:02, 588.33it/s]

356680 삭제


 51%|█████     | 1644/3241 [00:05<00:03, 504.08it/s]

442900 삭제
446150 삭제


100%|██████████| 3241/3241 [00:06<00:00, 490.83it/s]

005110 삭제
005180 삭제
005250 삭제
005257 삭제
005300 삭제
005305 삭제
005320 삭제
005360 삭제
005380 삭제
005385 삭제
005387 삭제
005389 삭제
005390 삭제
005420 삭제
005430 삭제
005440 삭제
005490 삭제
005500 삭제
005610 삭제
005680 삭제
005690 삭제
005720 삭제
005725 삭제
005740 삭제
005745 삭제
005750 삭제
005800 삭제
005810 삭제
005820 삭제
005830 삭제
005850 삭제
005870 삭제
005880 삭제
005930 삭제
005935 삭제
005940 삭제
005945 삭제
005950 삭제
005960 삭제
005965 삭제
006040 삭제
006060 삭제
